In [17]:
import spacy


# Cargar el modelo de lenguaje de spaCy
nlp = spacy.load("en_core_web_lg")

def analyze_sentiment(text):
    # Procesar el texto con spaCy
    doc = nlp(text)
    
    # Obtener el puntaje de polaridad del análisis de sentimiento
    sentiment_score = doc.sentiment.polarity
    
    # Definir umbrales para clasificar el sentimiento
    if sentiment_score <= -0.2:
        return 0  # Sentimiento negativo
    elif sentiment_score >= 0.2:
        return 2  # Sentimiento positivo
    else:
        return 1  # Sentimiento neutral

# Ejemplo de uso
sample_text = "Este es un ejemplo de análisis de sentimiento con spaCy. ¡Me encanta!"
sentiment_label = analyze_sentiment(sample_text)
print(f"Análisis de sentimiento: {sentiment_label}")


AttributeError: 'float' object has no attribute 'polarity'

In [9]:
# the following installations are required
# python -m textblob.download_corpora
# python -m spacy download en_core_web_sm
import nltk
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.blob.polarity                            # Polarity: -0.125
doc._.blob.subjectivity                        # Subjectivity: 0.9
doc._.blob.sentiment_assessments.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
doc._.blob.ngrams()                            # [WordList(['I', 'had', 'a']), WordList(['had', 'a', 'really']), WordList(['a', 'really', 'horrible']), WordList(['really', 'horrible', 'day']), WordList(['horrible', 'day', 'It']), WordList(['day', 'It', 'was']), WordList(['It', 'was', 'the']), WordList(['was', 'the', 'worst']), WordList(['the', 'worst', 'day']), WordList(['worst', 'day', 'ever']), WordList(['day', 'ever', 'But']), WordList(['ever', 'But', 'every']), WordList(['But', 'every', 'now']), WordList(['every', 'now', 'and']), WordList(['now', 'and', 'then']), WordList(['and', 'then', 'I']), WordList(['then', 'I', 'have']), WordList(['I', 'have', 'a']), WordList(['have', 'a', 'really']), WordList(['a', 'really', 'good']), WordList(['really', 'good', 'day']), WordList(['good', 'day', 'that']), WordList(['day', 'that', 'makes']), WordList(['that', 'makes', 'me']), WordList(['makes', 'me', 'happy'])]


**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt/english.pickle

  Searched in:
    - 'C:\\Users\\leabr/nltk_data'
    - 'c:\\Users\\leabr\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'c:\\Users\\leabr\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'c:\\Users\\leabr\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\leabr\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************



MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


In [31]:
from textblob import TextBlob

def analyze_sentiment(text):
    # Crear un objeto TextBlob con el texto
    blob = TextBlob(text)
    
    # Obtener la polaridad del sentimiento (-1 a 1, donde -1 es negativo, 0 es neutral y 1 es positivo)
    sentiment_score = blob.sentiment.polarity
    
    # Definir umbrales para clasificar el sentimiento
    if sentiment_score <= -0.2:
        return 0, sentiment_score  # Sentimiento negativo
    elif sentiment_score >= 0.04:
        return 2, sentiment_score  # Sentimiento positivo
    else:
        return 1, sentiment_score  # Sentimiento neutral

# Ejemplo de uso
sample_text = "Bad,people just spam 1 op character 1/10 "
sentiment_label = analyze_sentiment(sample_text)
print(f"Análisis de sentimiento: {sentiment_label}")


Análisis de sentimiento: (1, 0.0)


In [20]:
blob.sentiment.polarity

NameError: name 'blob' is not defined

In [32]:
import pandas as pd
from ast import literal_eval

# Cargar los archivos CSV
steam_games = pd.read_csv('steam_games.csv')
user_items = pd.read_csv('user_items.csv', converters={'items': literal_eval})

# Función para obtener el año con más horas jugadas para un género dado
def PlayTimeGenre(genero):
    # Filtrar los juegos por el género proporcionado
    juegos_por_genero = steam_games[steam_games['genres'].apply(lambda x: genero in x)]

    # Inicializar un diccionario para almacenar las horas jugadas por año
    horas_por_año = {}

    # Iterar sobre los juegos del género
    for index, juego in juegos_por_genero.iterrows():
        # Obtener el ID del juego
        juego_id = juego['id']

        # Filtrar las entradas correspondientes en user_items
        entradas_juego = user_items[user_items['items'].apply(lambda x: any(item.get('item_id') == juego_id for item in x))]

        # Iterar sobre las entradas del juego
        for _, entrada in entradas_juego.iterrows():
            # Obtener el año del juego
            año_juego = pd.to_datetime(juego['release_date']).year

            # Obtener las horas jugadas
            horas_jugadas = sum(item.get('playtime_forever', 0) for item in entrada['items'] if item.get('item_id') == juego_id)

            # Agregar las horas jugadas al diccionario por año
            if año_juego in horas_por_año:
                horas_por_año[año_juego] += horas_jugadas
            else:
                horas_por_año[año_juego] = horas_jugadas

    # Encontrar el año con más horas jugadas
    año_mas_horas = max(horas_por_año, key=horas_por_año.get)

    return año_mas_horas

# Ejemplo de uso
genero_consultado = 'Action'
año_resultado = PlayTimeGenre(genero_consultado)
print(f"Para el género '{genero_consultado}', el año con más horas jugadas es {año_resultado}.")


TypeError: argument of type 'float' is not iterable

In [33]:
import pandas as pd
from ast import literal_eval

# Cargar el archivo CSV
user_items = pd.read_csv('user_items.csv', converters={'items': literal_eval})

# Inicializar un diccionario para almacenar la sumatoria de horas por item_id
sumatoria_horas_por_item = {}

# Iterar sobre las entradas del archivo user_items
for _, entrada in user_items.iterrows():
    # Iterar sobre los diccionarios en la columna 'items'
    for item in entrada['items']:
        # Obtener el item_id y las horas jugadas
        item_id = item.get('item_id')
        horas_jugadas = item.get('playtime_forever', 0)

        # Agregar las horas jugadas al diccionario por item_id
        if item_id in sumatoria_horas_por_item:
            sumatoria_horas_por_item[item_id] += horas_jugadas
        else:
            sumatoria_horas_por_item[item_id] = horas_jugadas

# Crear un DataFrame a partir del diccionario
sumatoria_horas_df = pd.DataFrame(list(sumatoria_horas_por_item.items()), columns=['item_id', 'sumatoria_horas'])

# Guardar el nuevo archivo CSV
sumatoria_horas_df.to_csv('sumatoria_horas_por_item.csv', index=False)


In [43]:
import pandas as pd

def PlayTimeGenre(genero: str):
    # Cargar el DataFrame resultante del proceso anterior
    result_grouped = pd.read_csv('PlayTimeGenre.csv')

    # Filtrar por el género específico
    genre_data = result_grouped[result_grouped['genres'] == genero]

    if not genre_data.empty:
        # Encontrar el año con más horas jugadas
        max_year = int(genre_data.loc[genre_data['sumatoria_horas'].idxmax(), 'year'])
        max_hours = genre_data['sumatoria_horas'].max()

        return f"Año de lanzamiento con más horas jugadas para Género '{genero}': {max_year}"
    else:
        return f"No hay información para el género '{genero}'."

# Ejemplo de uso
genero_buscar = 'Strategy'
resultado = PlayTimeGenre(genero_buscar)
print(resultado)


Año de lanzamiento con más horas jugadas para Género 'Strategy': 2013
